In [2]:
%pip install duckdb numpy pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 388.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 653.6 kB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 169.3 kB/s eta 0:00:0000:0100:03
   ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/12.8 MB 102.4 kB/s eta 0:01:41^C
   ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/12.8 MB 102.4 kB/s eta 0:01:41
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [3]:
import duckdb
import pandas as pd
import numpy as np


# -----------------------------------------
# MinIO / S3 configuration
# -----------------------------------------

AWS_ACCESS_KEY = "amin"
AWS_SECRET_KEY = "password"
S3_ENDPOINT    = "http://localhost:9000"
WAREHOUSE_PATH = "s3://warehouse/default"   # محل ذخیره Iceberg روی MinIO
TABLE          = "user_evens"            # اسم جدول Iceberg

# -----------------------------------------
# Connect DuckDB + S3 + Iceberg
# -----------------------------------------

con = duckdb.connect()

# فعال‌سازی S3 در DuckDB
con.execute(f"""
SET s3_url='{S3_ENDPOINT}';
SET s3_access_key_id='{AWS_ACCESS_KEY}';
SET s3_secret_access_key='{AWS_SECRET_KEY}';
SET s3_use_ssl=false;
SET s3_endpoint='{S3_ENDPOINT.replace("http://", "").replace("https://", "")}';
SET enable_http_metadata_cache=true;
""")

# ثبت کاتالوگ Iceberg از MinIO
con.execute(f"""
CREATE OR REPLACE DATABASE iceberg_catalog
LOCATION '{WAREHOUSE_PATH}';
""")

print("Connected to Iceberg via MinIO successfully.")


# -----------------------------------------
# 1) خواندن داده‌ها از Iceberg
# -----------------------------------------

df = con.execute(f"""
    SELECT * 
    FROM iceberg_catalog.{TABLE}
""").df()

print("Data loaded from Iceberg:")
print(df.head())
print("\nRows:", len(df))


# -----------------------------------------
# 2) تحلیل‌های اولیه با Pandas / NumPy
# -----------------------------------------

print("\n--- Analysis ---")

# تعداد اکشن‌ها
action_counts = df["action"].value_counts()
print("\nAction distribution:")
print(action_counts)

# متوسط مدت زمان حضور (duration)
avg_duration = df["duration"].mean()
print("\nAverage duration:", avg_duration)

# پرکاربرترین 10 کاربر
top_users = df["user_id"].value_counts().head(10)
print("\nTop users:")
print(top_users)

# کاربرانی که بیشتر از 10 بار خرید یا checkout انجام داده‌اند
active_buyers = df[df["action"] == "checkout"]["user_id"].value_counts()
print("\nActive buyers:")
print(active_buyers.head(10))

# بخش‌هایی که بیشترین ترافیک را دارند (sections)
section_stats = df["section"].value_counts()
print("\nSection stats:")
print(section_stats)


# -----------------------------------------
# 3) مثال کوئری تحلیلی مستقیم با DuckDB
# -----------------------------------------

print("\n--- DuckDB SQL Analysis ---")

sql_result = con.execute(f"""
    SELECT action, COUNT(*) AS total
    FROM iceberg_catalog.{TABLE}
    GROUP BY action
    ORDER BY total DESC
""").df()

print(sql_result)

print("\nDone.")


ModuleNotFoundError: No module named 'pandas'